In [3]:
from core import game
import random
from Virtual_env import Virtual_Env
import numpy as np
import gym
from keras.layers import Conv2D,Dense, Flatten, Input, concatenate, Reshape, MaxPooling2D, Dropout, LayerNormalization
from keras.models import Sequential 
from keras.optimizers import Adam, SGD
from keras.models import Model
import keras
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
from IPython.display import display, clear_output

In [4]:
discount_factor = 0.95 
epsilon = 1 
epsilon_min = 0.01 
epsilon_interval = 0.99  
batch_size = 512  

env = Virtual_Env()

In [5]:
def create_q_model():
    input_all = Input(shape=(4), name = 'Input')

    # x1 = LayerNormalization()()
    x1 = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_all)
    # x1 = Dense(8, activation='relu')(x1)
    x1 = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x1)

    output = Dense(1, activation='linear')(x1)

    model = Model(inputs=input_all, outputs=output)

    return model

q_model = create_q_model()

current_state_memory = []
next_state_memory = []
reward_memory = []
done_memory = []
cumm_reward_history = []

episode_train_interval = 4

episode_count = 0
frame_count = 0

q_model.compile(loss = 'mse', optimizer='adam')

In [8]:
q_model.load_weights(r'C:\Users\Glazed\Documents\Root\Python\Tetris AI\Model_Weights.h5')

In [ ]:
import time
time.sleep(1)

while True: 
    state = env.reset()

    cumm_reward = 0
    done = False
    while not done:
        frame_count += 1

        next_possible_states = np.array([np.array(i) for i in env.getNextStates(env.piece).keys()])

        if np.random.uniform() < epsilon:
            best_move = next_possible_states[np.random.choice(next_possible_states.shape[0], size=1), :].reshape((20,10))
        else:
            best_value = None
            for poss_state in next_possible_states:
                value = q_model.predict(env.pg2state(poss_state.reshape((20, 10))).reshape(1, 4), verbose = 0)
                if not best_value or value > best_value:
                    best_value = value
                    best_move = poss_state

        best_action = env.getNextStates(env.piece)[tuple(best_move.flatten())]
        old_pg = env.playground
        next_state, reward, done, _  = env.step(best_action)

        new_pg = env.playground
        cumm_reward += reward


        current_state_memory.append(state)
        next_state_memory.append(next_state)
        reward_memory.append(reward)
        done_memory.append(done)        

        state = next_state

    cumm_reward_history.append(cumm_reward)
    episode_count += 1
    print("Currently on episode: %s with reward %s\n"%(episode_count, cumm_reward))
    if episode_count % episode_train_interval == 0:
        x = []
        y = []

        if len(current_state_memory) < batch_size:
            amount_to_sample = len(current_state_memory)
        else:
            amount_to_sample = batch_size

        random_batch = np.random.choice(len(current_state_memory),amount_to_sample , replace=False)

        batch_current_memory = np.array(current_state_memory)[random_batch]
        batch_next_memory = np.array(next_state_memory)[random_batch]
        batch_reward_memory = np.array(reward_memory)[random_batch]
        batch_done_memory = np.array(done_memory)[random_batch]

        next_predicted_q_values = q_model.predict(np.array(batch_next_memory))
        for current, reward, done, next_q in zip(batch_current_memory, batch_reward_memory, batch_done_memory, next_predicted_q_values):
            if not done:
                new_q_value = (reward + discount_factor * next_q)[0]
            else:
                new_q_value = reward

            x.append(current)
            y.append(new_q_value)

        q_model.fit(np.array(x), np.array(y), batch_size=batch_size)

        if epsilon > epsilon_min:
            epsilon -= epsilon_interval

    keras.backend.clear_session()
    q_model.reset_states()

    if (len(current_state_memory) > 8192):
        del current_state_memory[0]
        del next_state_memory[0]
        del reward_memory[0]
        del done_memory[0]

    if len(cumm_reward_history) > 8192:
        del cumm_reward_history[0]

In [10]:
q_model.save_weights('Model_Weights.h5')